In [12]:
import os
import cv2
import numpy as np
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.optimizers import Adam

In [13]:
DATADIR = "dataset/train/"
CATEGORIES = ["lajthiza", "spring", "tepelena"]

# Defining the size of the images expected by MobileNetV2
IMG_SIZE = 224

# Loading images directly from the dataset folder and preprocess them
def load_images():
    X = []
    y = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            if img.endswith(".jpg"):  # Ensure only .jpg files are processed
                try:
                    img_path = os.path.join(path, img)
                    image = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                    image = img_to_array(image)
                    image = preprocess_input(image)
                    X.append(image)
                    y.append(class_num)
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")
    return np.array(X), np.array(y)

# Loading the images and labels
X, y = load_images()

# Verifying the shape of the loaded data
print(f"Loaded images shape: {X.shape}")
print(f"Loaded labels shape: {y.shape}")


Loaded images shape: (969, 224, 224, 3)
Loaded labels shape: (969,)


In [14]:
num_classes = len(np.unique(y))  # Automatically determine number of classes
y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

# Spliting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Verifying data shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (775, 224, 224, 3)
y_train shape: (775, 3)
X_val shape: (194, 224, 224, 3)
y_val shape: (194, 3)


In [17]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freezing the base model
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluating the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 720ms/step - accuracy: 0.6574 - loss: 0.8425 - val_accuracy: 1.0000 - val_loss: 0.0281
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 695ms/step - accuracy: 0.9874 - loss: 0.0538 - val_accuracy: 0.9948 - val_loss: 0.0225
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 838ms/step - accuracy: 0.9858 - loss: 0.0335 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 681ms/step - accuracy: 1.0000 - loss: 0.0122 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 628ms/step - accuracy: 0.9979 - loss: 0.0094 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 628ms/step - accuracy: 0.9998 - loss: 0.0048 - val_accuracy: 1.0000 - val_loss: 0.0024
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 607ms/step - accuracy: 0.9976 - loss: 0.0106 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 584ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accu

In [19]:
model.save('water_bottle_classifier_mobilenetv2_pretrained.h5')